In [7]:
import json
from datetime import datetime, timedelta
from collections import defaultdict

def condense_geojson_to_weekly(input_file, output_file):
    # Load the GeoJSON data
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Organize data by station and week
    weekly_data = defaultdict(lambda: {
        "sum_ridership": 0,
        "latitude": None,
        "longitude": None
    })

    for feature in data["features"]:
        properties = feature["properties"]
        geometry = feature["geometry"]

        station_name = properties["Station"]
        ridership = properties["Ridership"]
        date = properties["Time"]
        coordinates = geometry["coordinates"]

        # Parse the date to determine the start of the week (Monday as the start)
        try:
            parsed_date = datetime.strptime(date, '%m/%d/%Y')
            start_of_week = (parsed_date - timedelta(days=parsed_date.weekday())).strftime('%m/%d/%Y')
        except ValueError as e:
            print(f"Error parsing date '{date}': {e}")
            continue  # Skip invalid dates

        # Aggregate data
        key = (station_name, start_of_week)
        weekly_data[key]["sum_ridership"] += ridership
        weekly_data[key]["latitude"], weekly_data[key]["longitude"] = coordinates

    # Create new GeoJSON structure
    weekly_features = []

    for (station_name, start_of_week), values in weekly_data.items():
        weekly_features.append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [values["longitude"], values["latitude"]]
            },
            "properties": {
                "Station": station_name,
                "Time": start_of_week,  # The start of the week
                "Ridership": values["sum_ridership"]
            }
        })

    weekly_collection = {
        "type": "FeatureCollection",
        "features": weekly_features
    }

    # Save the condensed GeoJSON file
    with open(output_file, 'w') as f:
        json.dump(weekly_collection, f, indent=2)

# Example usage
input_geojson = 'Subway_ridership_true_final_formatted.geojson'
output_geojson = 'Subway_ridership_weekly.geojson'
condense_geojson_to_weekly(input_geojson, output_geojson)
